# Vector elements in 2D


The finite element spaces we have seen so far, such as  the Lagrange, the DG, and the nonconforming finite element spaces, are all spaces of *scalar*-valued functions. How can we approximate  *vector fields*?  We could certainly use a Cartesian product of  scalar finite element spaces to approximate a vector field component by component. However, better properties can often be obtained using tailored vector finite element spaces, such as the *Raviart-Thomas* (RT) space or the *Nedelec* space. In this notebook we see how to access these spaces in ngsolve and compute with them.

## The Raviart-Thomas space

$\newcommand{\om}{\varOmega}
\newcommand{\oh}{\varOmega_h}
\newcommand{\dive}{\mathop{\mathrm{div}}}
\newcommand{\grad}{\mathop{\mathrm{grad}}}
\renewcommand{\div}{\mathop{\mathrm{div}}}
\newcommand{\R}{\mathbb{R}}
\newcommand{\jump}{\mathop{\mathrm{jump}}}
$
Consider a facet $F$ shared by two adjacent elements $K^\pm$ in two-dimensional mesh $\oh$ of a domain $\om$. In a [previous](BestApprox.ipynb) notebook, we examined jumps of a scalar piecewise function $w$ on the mesh, written as &LeftDoubleBracket; $ w$ &RightDoubleBracket; or simply $\jump(w)$. Here, we consider jumps of the normal component of a piecewise smooth *vector* field $q: \om \to \R^2$ across the facet $F$. Define &LeftDoubleBracket; $q\cdot n$ &RightDoubleBracket;, also denoted by $\jump(q\cdot n)$, by 

$$\tag{1}
\jump(q \cdot n)|_F := \Big(q|_{K^+} \cdot n_+ + q|_{K^-} \cdot n_-\Big) \Big|_F,
$$

where $n^\pm$ denote the outward unit normal on the boundary of the element $K^\pm$. Since the sign of $n_+$  and $n_-$ are opposite on $F$, this indeed measures the discontinuity in the normal component of $q$. This definition, facet by facet, defines the *normal jump* function $\jump(q \cdot n)$ of the vector field $q$ on the union of all interior facets. (Note that this jump definition if independent of the facet orientation.)

```{index} jump; of normal component
```

```{index} jump; orientation-independent definition
```

The Raviart-Thomas (RT) finite element space on a two-dimensional mesh $\newcommand{\oh}{\varOmega_h}$ $\oh$ (of a domain $\newcommand{\om}{\varOmega} \om$) is defined by 

$$
\begin{aligned}
R_{hp} =
\Big\{
q : \; \newcommand{\jump}{\mathop{\mathrm{jump}}}
 & q|_K \in P_p(K)^2 + \begin{pmatrix} x \\ y \end{pmatrix} P_p(K) \;
 \text{ on all mesh elements } K, 
\\ 
&\text{ and } \jump(q\cdot n) =0  \text{ on all interior mesh facets}
 \Big\}
\end{aligned}
$$

for any $p \ge 0$. Here, $P_p(K)$ denote the set of polynomials of degree at most $p$. From the definition, it is clear  that  plots of any function in the RT space should show a possibly discontinuous vector field with a *continuously varying normal component* across the interior mesh facets.

The continuity of the normal component is a useful property to have when we know that the  function being approximated has that same normal continuity property. *Fluxes*, such as the flux of a fluid flow, current flow, or magnetic flux,  are examples of vector fields whose normal components  remain continuous across a material interface, even when its other components jump across the interface.

### Approximation

Consider approximating the following vector field  using the RT space:  

$$
q = 
\left\{
\begin{aligned}
& (\sin(x), y),  && x \le 1/2,
\\
& (\sin(x), 1-y), && x > 1/2.
\end{aligned}
\right.
$$

Across the interface $x=1/2$, its normal component (the $x$-component in this case) is continuous, while its tangential component is discontinuous. Let's plot $q$ after making two subdomains to the left and right of the $x=1/2$ line. 

In [1]:
import ngsolve as ng
import numpy as np
from ngsolve.webgui import Draw
from netgen.occ import OCCGeometry ,WorkPlane, Rectangle, Glue
from ngsolve import x, y, sin, cos, GridFunction, div, curl

wp = WorkPlane()  # Make left and right subdomains
lft = wp.Rectangle(1/2, 1).Face()
rgt = wp.MoveTo(1/2,0).Rectangle(1/2, 1).Face()
lft.faces.name = 'lft'; rgt.faces.name = 'rgt'

geo = Glue([lft,rgt])
Draw(geo);
mesh = ng.Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=1/4))

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

We make the needed piecewise constant coefficient function using the `MaterialCF` method in ngsolve.

```{index} piecewise constant; subdomain by subdomain
```

In [2]:
q = mesh.MaterialCF({'lft': (sin(x), y),   # Piecewise q on the left & right subdomains
                     'rgt': (sin(x), 1-y)})

The normal component (which is the $x$ component in this case) of $q$ is smooth:

In [3]:
Draw(q[0], mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

The tangential component of $q$ is discontinuous across the middle interface.

In [4]:
Draw(q[1], mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

We can also plot $q$ as a vector field.

In [5]:
Draw(q, mesh, vectors={'grid_size': 15});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

If we try to interpolate such a function using a product of Lagrange spaces, then the discontinuity at the interface will generate large interpolation errors. (We have [already seen](BestApprox.ipynb) a similar phenomena around discontinuities of scalar function approximation.)  Instead, knowing that the normal component is continuous, we can try to approximate the function into the RT space.  Such an approximation *liberates its tangential component from continuity constraints, while preserving the normal continuity.*


The implementation of the RT space $R_{hp}$ in NGSolve is accessed using `ng.HDiv`. The `Set` method, as in other finite element spaces in NGSolve, can be used to perform the Oswald approximation. The procedure for Oswald approximation, [already described previously](OneDim.ipynb), is the same for all finite element spaces: one first $L_2$-projects (component by component) the (vector) function to be approximated into each local finite element space, and then assigns to every coupling degree of freedom the average of the degrees of freedom of the projection that were fused to form that coupling degree of freedom.

```{index} Oswald approximation; vector elements
```

```{index} Raviart-Thomas space; HDiv
```

The $p$ in the next code cell indicates the polynomial degree in the definition of $R_{hp}$ above. Note that the lowest order case is $p=0$.

In [6]:
p = 5   # This is the p in the definition of R_hp above
R = ng.HDiv(mesh, order=p, RT=True)

In [7]:
qh = GridFunction(R)
qh.Set(q)
print('Error in RT interpolation =', 
      ng.sqrt(ng.Integrate((q - qh)**2, mesh)))

Error in RT interpolation = 2.4319667352337606e-11


Compare this to what happens when we interpolate the same function in the product of two Lagrange finite element spaces.

In [8]:
V2 = ng.VectorH1(mesh, order=p)  # Cartesian product of Lagrange spaces
qq = GridFunction(V2)
qq.Set(q)
print('Error in product Lagrange space interpolation =', 
      ng.sqrt(ng.Integrate((q - qq)**2, mesh)))

Error in product Lagrange space interpolation = 0.11081020823065106


Clearly the error in the product space approach is much higher than that of the error produced by the RT space. The source of the large error while interpolating using  `ng.VectorH1` is immediately evident if you plot the $y$-component of the interpolant `qq` or the error.  The interpolant, being continuous in both components, has a difficult time approximating the discontinuous component, as seen from the plot of the error below.

In [9]:
Draw(qq - q, mesh, 'Lagrange interpolation error', vectors={'grid_size': 25});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

### Shape functions

It is illustrative to visualize the basis functions of the RT space. Recall that basis functions dual to the set of degrees of freedom are called shape functions. Since we cannot visualize the degrees of freedom functionals, visualizing shape functions is one way to understand what degrees of freedom are implemented in a code.  Here is a visualization of a shape function of the *lowest order RT space* (the $p=0$ case).

In [10]:
R = ng.HDiv(mesh, order=0, RT=True)

shapenumber = 15
shape = ng.GridFunction(R, name='shape')
shape.vec[:] = 0
shape.vec[shapenumber] = 1
Draw(shape, mesh, vectors={'grid_size': 20});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

You can see any shape function you wish by revising the `shapenumber` in the code above. In all cases, you see that
- the shape functions are vector fields whose normal components are continuous, 
- they are supported on a patch of one or two elements, 
- their normal component is zero on all mesh facets except one.


Thus in the lowest order case, each shape function is associated to a single mesh facet (an edge,  in our 2D mesh). If this edge is on the domain boundary, the  shape function is supported on one triangle. If the edge is an interior edge, the  shape function is supported on the two triangles which share the interior edge.

One can also directly see the connection between the number of mesh facets and the dimension of the lowest order RT space by querying both numbers:

In [11]:
mesh.nfacet

50

In [12]:
R.ndof

50

In [13]:
el0 = mesh[ng.NodeId(ng.ELEMENT, 0)]
type(el0)

ngsolve.comp.MeshNode

In [14]:
mesh.nedge

50

In [15]:
from ngsolve import EDGE, NodeId
e = NodeId(EDGE, mesh.nedge-1)
meshe = mesh[e] 
meshe.elements

(<ngsolve.comp.ElementId at 0x12151db30>,
 <ngsolve.comp.ElementId at 0x12152c9b0>)

In [16]:
print(meshe.elements)

(<ngsolve.comp.ElementId object at 0x12151d630>, <ngsolve.comp.ElementId object at 0x12151e970>)


In [17]:
Rd = ng.Discontinuous(ng.HDiv(mesh, order=0, RT=True))

In [18]:
Rd.GetDofNrs(meshe.elements[0]), Rd.GetDofNrs(meshe.elements[1])

((57, 58, 59), (63, 64, 65))

In [19]:
shapenumber = 1
shape = ng.GridFunction(R, name='shape')
shape.vec[:] = 0
shape.vec[shapenumber] = 1
Draw(shape, mesh, vectors={'grid_size': 20});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [20]:
shapenumber = 4
shape = ng.GridFunction(R, name='shape')
shape.vec[:] = 0
shape.vec[shapenumber] = 1
Draw(shape, mesh, vectors={'grid_size': 20});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

Of course, for higher $p$, a number of additional shape functions must be added. You can easily visualize the higher order shape functions by tweaking the above few lines of code.

### Convergence of divergence

Ordinarily, when a piecewise polynomial approximation of a function $q$ converges in $L_2$ at some rate, we expect its derivatives to converge at a lower rate. We already saw this rate reduction in a [prior notebook](Rates.ipynb).

However,  somewhat miraculously, the Raviart-Thomas finite element interpolant $I_h^{RT} q$ has the property that $\newcommand{\dive}{\mathrm{div}} \dive(I_h^{RT} q - q)$ converges at the same rate as $I_h^{RT} q - q$.  To obtain this interpolant in NGSolve, we use the `Set` method with `dual=True` option. We then compute the errors on successively refined meshes and tabulate the convergence rates much the same way as in a [previous notebook](B_ProjectionBAE.ipynb).

In [26]:
from prettytable import PrettyTable

def InterpolateOnSuccessiveRefinements(q, divq, mesh0, p=0, nrefinements=8):                                   
    """Error in RT interpolant on a sequence of uniformly refined meshes."""
      
    errors = []; diverrors = []; 
    mesh = ng.Mesh(mesh0.ngmesh.Copy())
    
    for ref in range(nrefinements): 
        RT0 = ng.HDiv(mesh, order=p , RT=True)
        qh = GridFunction(RT0)
        qh.Set(q, dual=True)   # Gives the canonical interpolant
        
        err = ng.sqrt(ng.Integrate((q - qh)**2, mesh))
        diverr = ng.sqrt(ng.Integrate((divq - div(qh))**2, mesh))
        errors.append(err); diverrors.append(diverr)      
        mesh.ngmesh.Refine()

    return np.array(errors), np.array(diverrors)

def TabulateRate(name, dat, h0=1):
    col = ['h', name, 'rate'];  h0col = ['%g'%h0]
    t = PrettyTable()
    t.add_column(col[0], h0col + [h0col[0] + '/' + str(2**i) for i in range(1, len(dat))])
    t.add_column(col[1], ['%.12f'%e for e in dat])
    t.add_column(col[2], ['*'] +  ['%1.2f' % r for r in np.log(dat[:-1]/dat[1:])/np.log(2)])
    print(t)

To obtain the convergence tables, we call these functions with the above set discontinuous vector field $q$. By the formula for distributional divergence of discontinuous fields from the lectures, we find that $\dive(q)$ has no edge distributions and can simply be given piecewise as follows:

In [22]:
divq = mesh.MaterialCF({'lft': cos(x)+1, 'rgt': cos(x)-1})

Using `q` and `divq`, we compute the errors and tabulate the rates:

In [23]:
err, diverr = InterpolateOnSuccessiveRefinements(q, divq, mesh)
TabulateRate('||q - IhRT(q)||', err, h0=1/4)
TabulateRate('||div(q - IhRT(q))||', diverr, h0=1/4)

+----------+-----------------+------+
|    h     | ||q - IhRT(q)|| | rate |
+----------+-----------------+------+
|   0.25   |  0.069664540643 |  *   |
|  0.25/2  |  0.037147230392 | 0.91 |
|  0.25/4  |  0.019223329290 | 0.95 |
|  0.25/8  |  0.009781789379 | 0.97 |
| 0.25/16  |  0.004934318342 | 0.99 |
| 0.25/32  |  0.002478122438 | 0.99 |
| 0.25/64  |  0.001241815197 | 1.00 |
| 0.25/128 |  0.000621597722 | 1.00 |
+----------+-----------------+------+
+----------+----------------------+------+
|    h     | ||div(q - IhRT(q))|| | rate |
+----------+----------------------+------+
|   0.25   |    0.184130711628    |  *   |
|  0.25/2  |    0.179541241866    | 0.04 |
|  0.25/4  |    0.141028834747    | 0.35 |
|  0.25/8  |    0.104310336249    | 0.44 |
| 0.25/16  |    0.075322310053    | 0.47 |
| 0.25/32  |    0.053804209711    | 0.49 |
| 0.25/64  |    0.038235769046    | 0.49 |
| 0.25/128 |    0.027103820642    | 0.50 |
+----------+----------------------+------+


**Questions for discussion:**

- What is the rate if you use the Oswald approximation instead of the canonical RT interpolant?
- Is the divergence computed in `divq` above 

## The Nédélec finite element space

Consider the curl operator in two space dimensions, acting on a vector field $u \equiv \begin{bmatrix}u_0 \\ u_1 \end{bmatrix}: \varOmega \to \mathbb R^2$, 

$$
\newcommand{\curl}{\mathop{\mathrm{curl}}}
\curl u = \frac{\partial u_1}{\partial x}  - \frac{\partial u_0}{\partial y}.
$$

You have  worked with $\curl$ as an operator acting on three-dimensional (3D) vector fields. The above two-dimensional (2D) version is the obtained as the $z$-component of the 3D curl when applied to a vector field that has only $x$ and $y$ components with no $z$ dependence. It takes a 2D vector field $u$ and produces a scalar function $\curl u$.

This 2D curl is intimately related to the 2D divergence. To see this, let $J_{\pi/2}$ denote the operator that rotates a vector clockwise by 90 degrees, i.e., 

$$
J_{\pi/2} 
\begin{pmatrix}
a \\ b
\end{pmatrix} = 
\begin{pmatrix}
b \\ -a
\end{pmatrix}.
$$

Now, if two vector fields $u$ and $q$ are related by  

$$
u = J_{\pi/2} q 
$$

then obviously 

$$ \newcommand{\dive}{\mathop{\mathrm{div}}}
\dive q =  \curl u.
$$

Moreover, if $t$ denotes the counterclockwise unit tangent vector on element boundaries, then it is related to the unit outward normal $n$ at the same boundary point by 

$$
J_{\pi/2} n = t.
$$

Hence 

$$
u\cdot t = q \cdot n.
$$

Thus the condition that   &LeftDoubleBracket; $q\cdot n$ &RightDoubleBracket; $= 0$
is equivalent to the condition that the *jump of the tangential component* of $u$ vanish, 
&LeftDoubleBracket; $u\cdot t$ &RightDoubleBracket; $= 0.$
The definition of the *tangential jump* &LeftDoubleBracket; $u\cdot t$ &RightDoubleBracket; on an interface $F = \partial K_+ \cap \partial K_-$ is completely analogous to how we defined the normal jump and just involves replacing $n_\pm$ in that definition (1) with $t_\pm$.

We define the  **Nédélec space in two dimensions** $N_{hp}$ as the rotated RT space, i.e., 

$$
N_{hp} := J_{\pi/2} R_{hp}.  
$$

It consists of piecewise polynomial vector fields that are *tangentially continuous*. Applying the rotation operator $J_{\pi/2}$ to the polynomial functions in our previous definition of $R_{hp}$, we also find that $N_{hp}$ can be equivalently described  by 

$$
\newcommand{\curl}{\mathop{\mathrm{curl}}}\newcommand{\om}{\varOmega}
\begin{aligned}
N_{hp} =
\Big\{
u : \; \newcommand{\jump}{\mathop{\mathrm{jump}}}
 & 
 u|_K \in P_p(K)^2 + \begin{pmatrix} -y \\ x \end{pmatrix} P_p(K)
 \text{ on all elements } K, 
\\ 
&\text{ and } \jump(u\cdot t) =0  \text{ on all interior mesh facets}
 \Big\}.
\end{aligned}
$$

Here, the vector $(-y, x)$ arises as the rotation of the vector $(x, y)$ used in the RT space definition.

Note that although the **2D** Nédélec space is isomorphic to the 2D Raviart-Thomas space (via $J_{\pi/2}$), the **3D** Nédélec space seen in the lectures is  truly different from the 3D Raviart-Thomas space. (You can be convinced of this by counting their dimensions.)

NGSolve provides an implementation of the Nedelec space in 2D and 3D accessible through `HCurl` as follows.

In [24]:
N = ng.HCurl(mesh, order=0, type1=True)

In [25]:
shapenumber = 15
shape = GridFunction(N, name='shape')
shape.vec[:] = 0
shape.vec[shapenumber] = 1
Draw(shape, mesh, vectors={'grid_size': 30});

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

As you can see, this shape function visually appears to be exactly the same shape function plotted earlier for the RT case, except for a rotation by 90 degrees.  Note how the tangential component of the shape function varies continuously across the edges. Just as the RT space was useful for interpolating vector fields with normal continuity, the Nédélec  space is useful for interpolating vector fields with tangential continuity. 

The Nédélec  finite element is sometimes called the **edge finite element** in some engineering literature. Interestingly, the shape functions of the RT and the Nedelec element were known in another mathematical context (they appeared in a 1957 book by Whitney) long before these elements were known. In recognition of this, mathematicians sometimes refer to these shape functions as **Whitney forms**: what we have plotted above are indeed some scalar multiples of Whitney forms.  Their expressions in terms of barycentric coordinates are left as exercises.  

```{index} Whitney forms
```

**Questions for discussion:**

- Would the 2D curl of the canonical Nédélec interpolant superconverge? 

## Summary

We have seen

- the RT space with normal continuity,
- the Nédélec space with tangential continuity,
- that in 2D, one is obtained by a rotation of the other,
- the scalar 2D curl operator and its relation to divergence,
- how to interpolate vector fields with normal or tangential continuity using these spaces,
- the shape functions of these spaces,
- the superconvergence of divergence 